In [1]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

In [2]:
num_periods = 24

In [3]:
path_folder_processed = Path.cwd() / "data" / "input" / "processed"
# from 2019-01-01T00 to 2022-12-31 # hourly # leap date of 2020 02 29 removed 
# all years have 8760 hours
timestamp_hist = np.load(path_folder_processed / "timestamp_hist.npy")
renewable_cap_hist = np.load(path_folder_processed / "renewable_cap_hist.npy")
renewable_gen_hist = np.load(path_folder_processed / "renewable_gen_hist.npy")
renewable_ratio_hist = np.load(path_folder_processed / "renewable_ratio_hist.npy")
demand_hist = np.load(path_folder_processed / "demand_hist.npy")

In [4]:
def idx_time(time_str):
    return int(np.where(timestamp_hist == np.datetime64(time_str))[0][0])

In [5]:
idx_target_start, idx_target_end = idx_time("2022-07-21T00"), idx_time("2022-07-21T23")

In [6]:
renewable_cap_hist

array([2636.54    , 2636.54    , 2636.54    , ..., 6915.137055,
       6915.137055, 6915.137055], shape=(35040,))

In [7]:
# now from 2019 ~ 21
# SINGLE TIME RUN IN IPYNB 
renewable_cap_hist = renewable_cap_hist[:idx_time("2022-01-01")]
renewable_gen_hist = renewable_gen_hist[:idx_time("2022-01-01")]
renewable_ratio_hist = renewable_ratio_hist[:idx_time("2022-01-01")]
demand_hist = demand_hist[:idx_time("2022-01-01")]
timestamp_hist = timestamp_hist[:idx_time("2022-01-01")]

In [8]:
# ok i told gpt how to do it exactly the ideas are from mine 99% and i checked cus i can code well
from datetime import datetime


# helper → returns True if ts belongs in the ±30-day weekday window of year `yr`
def in_window_weekday(ts: datetime, yr: int) -> bool:
    target = datetime(yr, 7, 21)  # July-21 of that year
    delta = abs((ts.date() - target.date()).days)
    return delta <= 30 and ts.weekday() < 5  # ±30 days & Mon-Fri


arrays_demand = {}  # dict: year → (N_y,24)
arrays_ratio = {}

for yr in (2019, 2020, 2021):
    mask = np.array(
        [
            in_window_weekday(ts.astype("datetime64[s]").astype(datetime), yr)
            for ts in timestamp_hist
        ]
    )
    arrays_demand[yr] = demand_hist[mask].reshape(-1, 24)  # (N_y,24)
    arrays_ratio[yr] = renewable_ratio_hist[mask].reshape(-1, 24)  # (N_y,24)

# quick sanity check
for yr, arr in arrays_demand.items():
    print(f"{yr}:  days = {arr.shape[0]}  (should be ≈43)")

2019:  days = 43  (should be ≈43)
2020:  days = 44  (should be ≈43)
2021:  days = 45  (should be ≈43)


In [9]:
arrays_demand[2019], arrays_demand[2020], arrays_demand[2021]  # shapes (N_2019,24) ...
arrays_ratio[2019], arrays_ratio[2020], arrays_ratio[2021] ; 

In [10]:
arrays_demand[2019][:, 0].mean(), arrays_demand[2020][:, 0].mean(), arrays_demand[2021][:, 0].mean()
arrays_demand[2019][:, 0].min(), arrays_demand[2020][:, 0].min(), arrays_demand[2021][:, 0].min()
arrays_demand[2019][:, 0].max(), arrays_demand[2020][:, 0].max(), arrays_demand[2021][:, 0].max() ; 
# 2020 data.. omg also ok this is just first hour mid night 

In [11]:
t = 12
arrays_ratio[2019][:, t].mean(), arrays_ratio[2020][:, t].mean(), arrays_ratio[2021][:, t].mean()
arrays_ratio[2019][:, t].max(), arrays_ratio[2020][:, t].max(), arrays_ratio[2021][:, t].max()
arrays_ratio[2019][:, t].min(), arrays_ratio[2020][:, t].min(), arrays_ratio[2021][:, t].min() ; 

In [12]:
def ppp():
    import matplotlib.pyplot as plt
    import numpy as np
    import matplotlib.collections as mcoll

    # ------------- user-tunable global linewidth -----------------
    LW = 2          
    # -------------------------------------------------------------

    # assumes arrays_demand dict exists
    fig, ax = plt.subplots(figsize=(25, 10))
    hours  = np.arange(1, 25)
    colors = {2019: 'blue', 2020: 'green', 2021: 'red'}

    for yr in (2019, 2020, 2021):
        vp = ax.violinplot(
            # arrays_ratio[yr],
            arrays_demand[yr] / 1000,
            positions=hours,
            widths=0.8,
            showmeans=True,
            showmedians=False,
            showextrema=False
        )
        # outline (left half only)
        for body in vp['bodies']:
            body.set_facecolor('none')
            body.set_edgecolor(colors[yr])
            body.set_linewidth(LW)
            cx = body.get_paths()[0].vertices[:, 0].mean()
            body.set_clip_path(
                plt.Rectangle((cx-10, -1e6), 10, 2e6, transform=ax.transData)
            )
            body.set_alpha(1)

        # mean bar (right half only)
        means = vp['cmeans']
        means.set_color(colors[yr])
        means.set_linewidth(LW)
        segs = means.get_segments()
        new_segs = []
        for seg in segs:
            x0, y0 = seg[0]; x1, y1 = seg[1]
            mid = (x0 + x1)/2
            new_segs.append([[mid, y0], [x1, y1]])
        means.set_segments(new_segs)
        means.set_alpha(1)

    # cosmetics
    ax.set_xticks(hours)
    ax.set_xticklabels(hours)       # 1, 2, …, 24
    ax.set_xlim(0.5, 24.5)              # x-limits from 1 to 24

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    for s in ('bottom', 'left'):
        ax.spines[s].set_linewidth(LW)

    ax.tick_params(axis='both', which='major',
                direction='in', length=10,
                labelsize=20, width=LW)
    ax.grid(True, axis='x', alpha=1, lw=LW)

    plt.tight_layout()
    # ax.set_ylim(0)
    # enforce opaque in case anything slipped through
    for poly in ax.findobj(match=mcoll.PolyCollection):
        poly.set_alpha(1)
    for lc in ax.findobj(match=mcoll.LineCollection):
        lc.set_alpha(1)

    plt.show()


In [13]:
# ppp()

In [14]:
# # i just did it manually 2019 2021 swap with var name too
# # its bascially grouping with bin = 6 so each data points will have 7 too fixed per bin this can be menteiond in improvement
# # but like weekday x ~-30 ~+30 the time horizon 24h single day this is reasonable . and probaly better than season (although seasonal 3 months can increase datapoints but should be filtered if we actually do )
# # also this is too small but going below 2018 is problematic  basically linear extrapolation will be "accurate"
# # reneable ratio just ratio itself , and will be literally multipled by the capcity which is known (assume this probably true too)
# import numpy as np

# # pick whichever year you want, e.g. 2019:
# raw_demand = arrays_demand[2021][1:, :]    # shape (42,24)
# raw_ratio  = arrays_ratio [2021][1:, :]    # also (42,24)

# K        = 6
# bin_size = raw_demand.shape[0] // K        # 42//6 = 7

# # 1) sort days by some summary metric (e.g. total daily demand)
# order = np.argsort(raw_demand.sum(axis=1))

# # 2) form index‐sets B_k of size 7
# B = [order[k*bin_size:(k+1)*bin_size] for k in range(K)]

# # 3) compute centroids
# centroids_demand_2021 = np.stack(
#     [ raw_demand[Bk].mean(axis=0) for Bk in B ],
#     axis=1
# )   # shape (24,6), gives {}^{the}D^{sce}_{t,k}

# centroids_renratio_2021 = np.stack(
#     [ raw_ratio[Bk].mean(axis=0) for Bk in B ],
#     axis=1
# )   # shape (24,6) for ratio scenarios

# # 4) equal weights
# w = np.ones(K) / K     # array([1/6,1/6,…,1/6])


In [15]:
# np.save(path_folder_processed / "centroids_renratio_2019.npy", centroids_renratio_2019)
# np.save(path_folder_processed / "centroids_demand_2019.npy", centroids_demand_2019)
# np.save(path_folder_processed / "centroids_renratio_2021.npy", centroids_renratio_2021)
# np.save(path_folder_processed / "centroids_demand_2021.npy", centroids_demand_2021)
# basically above but loading it incase ipynb data corruption
centroids_demand_2021 = np.load(path_folder_processed / "centroids_demand_2021.npy")
centroids_renratio_2021 = np.load(path_folder_processed / "centroids_renratio_2021.npy")
centroids_demand_2019 = np.load(path_folder_processed / "centroids_demand_2019.npy")
centroids_renratio_2019 = np.load(path_folder_processed / "centroids_renratio_2019.npy")

In [16]:
# # i just exlcuded the  2020 its too bad i included it in the plot though in report 
# centroids_demand_2021.mean(axis=0)
# centroids_renratio_2021.mean(axis=0)
# centroids_demand_2019.mean(axis=0)
# centroids_renratio_2019.mean(axis=0)

In [17]:
# the linear increase (extrapolation) won't be too much so we good and its reasonable for each of 6 centroids
# i checked by .mean()
# #### note that ALTHOUGH ITS NAMED 2019 2021 2022 
# THESE ARE SPECIFIC TO OUR 7 21 TIME HORIZON
# we might have to compare these 6 scenarios with actual data but this isn't really the project focus anyways
# ok i changed to 1.5
centroids_demand_2022 = (centroids_demand_2021 - centroids_demand_2019) * 1.5+ centroids_demand_2019
# average for 2022 cus its ratio and doesn't make sense if we really think deep 
centroids_renratio_2022 = (centroids_renratio_2021 +  centroids_renratio_2019) /2 

In [18]:
# same reason
np.save(path_folder_processed / "centroids_demand_2022.npy", centroids_demand_2022)
np.save(path_folder_processed / "centroids_renratio_2022.npy", centroids_renratio_2022)
centroids_demand_2022= np.load(path_folder_processed / "centroids_demand_2022.npy")
centroids_renratio_2022= np.load(path_folder_processed / "centroids_renratio_2022.npy")

In [19]:
def asdf():
    # FUSING CENTROIDS WITH DISTRIBUTION 

    # ok i tried and it looked bad just too much stuffs in one plot so im gonna do seaparet just ceontroids lines 
    import matplotlib.pyplot as plt
    import numpy as np

    # ------------- user‐tunable global linewidth -----------------
    LW = 2
    # -------------------------------------------------------------

    # assumes centroids_demand_2019, centroids_demand_2021, centroids_demand_2022 exist
    fig, ax = plt.subplots(figsize=(25, 10))

    hours = np.arange(1, 25)
    colors = {
        2019: 'blue',
        2021: 'red',
        2022: 'black',
    }

    # plot centroid lines
    ax.plot(hours, centroids_demand_2019 / 1000, color=colors[2019], linewidth=LW)
    ax.plot(hours, centroids_demand_2021 / 1000, color=colors[2021], linewidth=LW)
    ax.plot(hours, centroids_demand_2022 / 1000, color=colors[2022], linewidth=LW)

    # cosmetics
    ax.set_xticks(hours)
    ax.set_xticklabels(hours)
    ax.set_xlim(0.5, 24.5)

    # remove top/right spines, thicken bottom/left
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    for s in ('bottom', 'left'):
        ax.spines[s].set_linewidth(LW)
    ax.set_yticks(range(50, 100, 10))
    # ticks
    ax.tick_params(
        axis='both', which='major',
        direction='in', length=10,
        labelsize=20, width=LW
    )
    # ax.set_ylim(0)


    # vertical grid lines only
    ax.grid(True, axis='x', alpha=1, linewidth=LW)

    plt.tight_layout()
    plt.show()


In [20]:
# asdf()

In [21]:
print(np.where(np.arange(np.datetime64("2022-01-01T00"), np.datetime64("2022-08-01T00")) == np.datetime64("2022-07-21T00"))[0][0])
# 4824 = 0h # 4847 = 23h 
target_rencap = np.load(path_folder_processed / "renewable_cap_2022.npy")[4824:4847 + 1]
print(np.all(target_rencap == 6322.68621))
target_rencap = 6322.68621 # so we just use this 

4824
True


In [22]:
centroids_rengen_2022 = centroids_renratio_2022 * 6322.68621
# np.save(path_folder_processed / "centroids_rengen_2022.npy", centroids_rengen_2022)
centroids_rengen_2022 = np.load(path_folder_processed / "centroids_rengen_2022.npy")
# plt.plot(centroids_rengen_2022) # 3 GW peak solar so looking good i guess

In [23]:
thermal_demand_2022721_tk = centroids_demand_2022 - centroids_rengen_2022

In [24]:
np.save(path_folder_processed / "thermal_demand_2022721_tk.npy", thermal_demand_2022721_tk)
# thermal_demand_2022721_tk = np.load(path_folder_processed / "thermal_demand_2022721_tk.npy")